In [1]:
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

import matplotlib.pyplot as plt

import os
import sys

from IPython.display import display, Image

from scipy import ndimage

import random
import cv2 
from sklearn.model_selection import train_test_split
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
%matplotlib inline

def show_img(img):
    fig = plt.gcf()
    fig.set_size_inches(18.5, 10.5)
    if len(img.shape) == 3:
        plt.imshow(img)
    else:
        img2 = img
        plt.imshow(img2, cmap='gray')
        plt.show()

# Get three channel data

In [2]:
%load_ext autoreload
%autoreload 2
from utils.preprocess_utils import *

In [4]:
shape = (64, 64)
channel = 3
data = generate_data_set_for_training(shape, channel)

dataset: (86345, 64, 64, 3)
labels: (86345, 6)
dataset: (13066, 64, 64, 3)
labels: (13066, 6)


In [5]:
X_train = data['X_train']
X_val = data['X_val']
y_train = data['y_train']
y_val = data['y_val']
X_test = data['X_test']
y_test = data['y_test']

# Try pretrianed Vgg

In [6]:
import numpy as np
import os
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras import regularizers
import keras.utils as ku
from keras import backend as K
from keras import optimizers
import tensorflow as tf
import pickle
import matplotlib.pyplot as ply

Using TensorFlow backend.


In [7]:
from utils.vgg16_utils import *

In [ ]:
# from scratch
run_pretrained_vgg16()

In [8]:
_, row, col, channel = X_train.shape
print('chanel', channel)
print('row', row)
print('col', col)

chanel 3
row 64
col 64


In [9]:
# Params
digLen = 5
numDigits = 11

In [10]:
preTrainModel = VGG16(include_top = False, weights = 'imagenet')
preTrainModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [11]:
ptInput = keras.Input(shape = (row,col,channel), name  = 'inputVGGPreTrain')
pt_vgg16 = preTrainModel(ptInput)

In [12]:
Mout = Flatten(name = 'flatten')(pt_vgg16)
Mout = Dense(1024, activation='relu', name = 'FC1_4096')(Mout)
Mout = Dense(1024, activation='relu', name = 'FC1_512')(Mout)

In [13]:
numd_SM = Dense(digLen,    activation='softmax',name = 'num')(Mout)
dig1_SM = Dense(numDigits, activation='softmax',name = 'dig1')(Mout)
dig2_SM = Dense(numDigits, activation='softmax',name = 'dig2')(Mout)
dig3_SM = Dense(numDigits, activation='softmax',name = 'dig3')(Mout)
dig4_SM = Dense(numDigits, activation='softmax',name = 'dig4')(Mout)
numB_SM = Dense(2, activation='softmax',name = 'nC')(Mout)
out = [numd_SM, dig1_SM ,dig2_SM, dig3_SM, dig4_SM,numB_SM] #numd_SM

In [14]:
    epochs = 50
    batch_size = 64
    lr = 0.001
    beta_1 = 0.9
    beta_2 = 0.999
    epsilon = None
    decay = 0.0
    amsgrad = True
    # defind optimizer



    optim = optimizers.Adam(lr=lr, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon, decay=decay, amsgrad=amsgrad)

In [15]:
vggPreTrain = keras.Model(inputs = ptInput, outputs = out)

vggPreTrain.compile(loss = 'sparse_categorical_crossentropy', #ceLoss ,
                    optimizer= optim,
                    metrics=  ['accuracy']) #[])
vggPreTrain.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputVGGPreTrain (InputLayer)   (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
vgg16 (Model)                   multiple             14714688    inputVGGPreTrain[0][0]           
__________________________________________________________________________________________________
flatten (Flatten)               (None, 2048)         0           vgg16[1][0]                      
__________________________________________________________________________________________________
FC1_4096 (Dense)                (None, 1024)         2098176     flatten[0][0]                    
__________________________________________________________________________________________________
FC1_512 (D

In [16]:
callback = []
checkpointer = keras.callbacks.ModelCheckpoint(filepath='saved_models/VGGPreTrained_classifier_corrected.hdf5',
                                               monitor='loss',
                                               save_best_only=True,
                                               verbose=2)

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'loss',
                                              factor = 0.1,
                                              verbose = 1,
                                              patience= 4,
                                              cooldown= 1,
                                              min_lr = 0.0001)
es = keras.callbacks.EarlyStopping(monitor= 'loss',
                                   min_delta=0.000001,
                                   patience=5,
                                   verbose=1,
                                   mode='auto')
callback.append(es)
callback.append(checkpointer)
callback.append(reduce_lr)

In [17]:
%%time
vggHistory = vggPreTrain.fit(x = X_train,
                             y = list(y_train.T),
                             batch_size = batch_size,
                             epochs=epochs,
                             verbose=1,
                             shuffle = True,
                             validation_data = (X_val, list(y_val.T)),
                             callbacks= callback)

print(vggHistory.history.keys())
modName = 'vgg16_PreTrain_corrected'
# list all data in history
print(vggHistory.history.keys())

Train on 69076 samples, validate on 17269 samples
Epoch 1/50
69076/69076 [==============================] - 91s 1ms/step - loss: 3.0098 - num_loss: 0.5095 - dig1_loss: 0.9262 - dig2_loss: 0.8843 - dig3_loss: 0.4488 - dig4_loss: 0.0972 - nC_loss: 0.1438 - num_acc: 0.8064 - dig1_acc: 0.6899 - dig2_acc: 0.7077 - dig3_acc: 0.8972 - dig4_acc: 0.9849 - nC_acc: 0.9468 - val_loss: 2.9973 - val_num_loss: 0.5401 - val_dig1_loss: 0.8658 - val_dig2_loss: 0.8990 - val_dig3_loss: 0.5226 - val_dig4_loss: 0.0920 - val_nC_loss: 0.0778 - val_num_acc: 0.7702 - val_dig1_acc: 0.7039 - val_dig2_acc: 0.6800 - val_dig3_acc: 0.8925 - val_dig4_acc: 0.9865 - val_nC_acc: 0.9721

Epoch 00001: loss improved from inf to 3.00977, saving model to saved_models/VGGPreTrained_classifier_corrected.hdf5
Epoch 2/50
69076/69076 [==============================] - 84s 1ms/step - loss: 2.3109 - num_loss: 0.2295 - dig1_loss: 0.8145 - dig2_loss: 0.7972 - dig3_loss: 0.3447 - dig4_loss: 0.0724 - nC_loss: 0.0526 - num_acc: 0.9150 - 

69076/69076 [==============================] - 85s 1ms/step - loss: 0.4782 - num_loss: 0.0495 - dig1_loss: 0.1287 - dig2_loss: 0.1633 - dig3_loss: 0.0952 - dig4_loss: 0.0277 - nC_loss: 0.0138 - num_acc: 0.9851 - dig1_acc: 0.9626 - dig2_acc: 0.9499 - dig3_acc: 0.9713 - dig4_acc: 0.9913 - nC_acc: 0.9957 - val_loss: 0.9018 - val_num_loss: 0.1331 - val_dig1_loss: 0.2461 - val_dig2_loss: 0.2967 - val_dig3_loss: 0.1546 - val_dig4_loss: 0.0380 - val_nC_loss: 0.0332 - val_num_acc: 0.9609 - val_dig1_acc: 0.9301 - val_dig2_acc: 0.9127 - val_dig3_acc: 0.9545 - val_dig4_acc: 0.9903 - val_nC_acc: 0.9907

Epoch 00012: loss improved from 0.48424 to 0.47821, saving model to saved_models/VGGPreTrained_classifier_corrected.hdf5
Epoch 13/50
69076/69076 [==============================] - 85s 1ms/step - loss: 0.4947 - num_loss: 0.0532 - dig1_loss: 0.1371 - dig2_loss: 0.1644 - dig3_loss: 0.0976 - dig4_loss: 0.0277 - nC_loss: 0.0147 - num_acc: 0.9837 - dig1_acc: 0.9599 - dig2_acc: 0.9495 - dig3_acc: 0.9702 -


Epoch 00023: loss improved from 0.22150 to 0.20470, saving model to saved_models/VGGPreTrained_classifier_corrected.hdf5
Epoch 24/50
69076/69076 [==============================] - 85s 1ms/step - loss: 0.1762 - num_loss: 0.0157 - dig1_loss: 0.0529 - dig2_loss: 0.0577 - dig3_loss: 0.0379 - dig4_loss: 0.0090 - nC_loss: 0.0031 - num_acc: 0.9950 - dig1_acc: 0.9842 - dig2_acc: 0.9819 - dig3_acc: 0.9873 - dig4_acc: 0.9970 - nC_acc: 0.9989 - val_loss: 0.6103 - val_num_loss: 0.0835 - val_dig1_loss: 0.1681 - val_dig2_loss: 0.1944 - val_dig3_loss: 0.1090 - val_dig4_loss: 0.0256 - val_nC_loss: 0.0298 - val_num_acc: 0.9801 - val_dig1_acc: 0.9625 - val_dig2_acc: 0.9558 - val_dig3_acc: 0.9732 - val_dig4_acc: 0.9929 - val_nC_acc: 0.9939

Epoch 00024: loss improved from 0.20470 to 0.17624, saving model to saved_models/VGGPreTrained_classifier_corrected.hdf5
Epoch 25/50
69076/69076 [==============================] - 85s 1ms/step - loss: 0.1694 - num_loss: 0.0138 - dig1_loss: 0.0506 - dig2_loss: 0.0574 

69076/69076 [==============================] - 85s 1ms/step - loss: 0.1524 - num_loss: 0.0151 - dig1_loss: 0.0464 - dig2_loss: 0.0496 - dig3_loss: 0.0308 - dig4_loss: 0.0066 - nC_loss: 0.0039 - num_acc: 0.9956 - dig1_acc: 0.9866 - dig2_acc: 0.9849 - dig3_acc: 0.9904 - dig4_acc: 0.9981 - nC_acc: 0.9990 - val_loss: 0.5756 - val_num_loss: 0.0742 - val_dig1_loss: 0.1607 - val_dig2_loss: 0.1819 - val_dig3_loss: 0.1073 - val_dig4_loss: 0.0254 - val_nC_loss: 0.0261 - val_num_acc: 0.9820 - val_dig1_acc: 0.9631 - val_dig2_acc: 0.9569 - val_dig3_acc: 0.9738 - val_dig4_acc: 0.9939 - val_nC_acc: 0.9940

Epoch 00035: loss did not improve from 0.12346
Epoch 36/50
69076/69076 [==============================] - 85s 1ms/step - loss: 0.1725 - num_loss: 0.0179 - dig1_loss: 0.0560 - dig2_loss: 0.0536 - dig3_loss: 0.0309 - dig4_loss: 0.0067 - nC_loss: 0.0073 - num_acc: 0.9948 - dig1_acc: 0.9836 - dig2_acc: 0.9839 - dig3_acc: 0.9903 - dig4_acc: 0.9981 - nC_acc: 0.9984 - val_loss: 0.5997 - val_num_loss: 0.08


Epoch 00046: loss improved from 0.02835 to 0.02496, saving model to saved_models/VGGPreTrained_classifier_corrected.hdf5
Epoch 47/50
69076/69076 [==============================] - 87s 1ms/step - loss: 0.0220 - num_loss: 0.0017 - dig1_loss: 0.0078 - dig2_loss: 0.0077 - dig3_loss: 0.0043 - dig4_loss: 3.9173e-04 - nC_loss: 1.3032e-04 - num_acc: 0.9996 - dig1_acc: 0.9980 - dig2_acc: 0.9980 - dig3_acc: 0.9989 - dig4_acc: 0.9999 - nC_acc: 1.0000 - val_loss: 0.7405 - val_num_loss: 0.1084 - val_dig1_loss: 0.1952 - val_dig2_loss: 0.2344 - val_dig3_loss: 0.1339 - val_dig4_loss: 0.0316 - val_nC_loss: 0.0370 - val_num_acc: 0.9823 - val_dig1_acc: 0.9664 - val_dig2_acc: 0.9621 - val_dig3_acc: 0.9794 - val_dig4_acc: 0.9955 - val_nC_acc: 0.9936

Epoch 00047: loss improved from 0.02496 to 0.02205, saving model to saved_models/VGGPreTrained_classifier_corrected.hdf5
Epoch 48/50
69076/69076 [==============================] - 87s 1ms/step - loss: 0.0196 - num_loss: 0.0015 - dig1_loss: 0.0070 - dig2_loss:

# vgg_pretrain_result

In [20]:
%load_ext autoreload
%autoreload 2
from utils.model_evaluation_utils import evaluate_model_performance

evaluate_model_performance(fitted_model=vggHistory, 
                           model_name = modName,
                           data=data,
                           model=vggPreTrain
                          )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Train loss: 0.014020312673659113
number_of_digits digit1 digit2 digit3 digit4
Train accuracy: [99.97538942613933, 99.88852857721929, 99.88708089640396, 99.93919740575598, 99.99855231918467, 99.99855231918467]
Train sequence accuracy: 99.74955121894725
Validation loss: 0.7995588414178265
number_of_digits digit1 digit2 digit3 digit4
Validation accuracy: [98.21066651224739, 96.65875267820951, 96.2302391568707, 97.91533962591927, 99.5425328623545, 99.36302044125311]
Validation sequence accuracy: 92.8716196652962
Test loss: 1.6288448797922923
number_of_digits digit1 digit2 digit3 digit4
Test accuracy: [96.75493647635084, 92.21643961426604, 91.71896525332926, 96.6630950558702, 99.58671360783713, 99.3418031532221]
Test sequence accuracy: 84.62421552120006


{'train_acc': [99.97538942613933,
  99.88852857721929,
  99.88708089640396,
  99.93919740575598,
  99.99855231918467,
  99.99855231918467],
 'test_acc': [96.75493647635084,
  92.21643961426604,
  91.71896525332926,
  96.6630950558702,
  99.58671360783713,
  99.3418031532221],
 'val_acc': [98.21066651224739,
  96.65875267820951,
  96.2302391568707,
  97.91533962591927,
  99.5425328623545,
  99.36302044125311],
 'train_seq_acc': 99.74955121894725,
 'test_seq_acc': 84.62421552120006,
 'val_seq_acc': 92.8716196652962,
 'train_score': [0.014020312673659113,
  0.0011900593736928655,
  0.00510851295568146,
  0.004846705962200736,
  0.002579063866183819,
  0.00021484057969230894,
  8.113011016274738e-05,
  0.9997538942613933,
  0.9988852857721929,
  0.9988708089640396,
  0.9993919740575598,
  0.9999855231918466,
  0.9999855231918466],
 'test_score': [1.6288448797922923,
  0.22731382164296057,
  0.524573706161919,
  0.5707254456551292,
  0.2350420507594548,
  0.02908931626101544,
  0.0421005515

In [17]:
# old

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Train loss: 0.0012191442427505605
number_of_digits digit1 digit2 digit3 digit4
Train accuracy: [100.0, 99.99710463836932, 99.99710463836932, 100.0, 100.0, 100.0]
Train sequence accuracy: 99.99420927673867
Validation loss: 0.8632769504238671
number_of_digits digit1 digit2 digit3 digit4
Validation accuracy: [98.65076148010886, 96.69928774103886, 95.89437720771325, 97.56789623023916, 99.35722971799179, 100.0]
Validation sequence accuracy: 92.33887312525334
Test loss: 1.9249597305217272
number_of_digits digit1 digit2 digit3 digit4
Test accuracy: [96.96157967243226, 91.51997550895454, 89.759681616409, 95.83652227154447, 99.44129802540947, 100.0]
Test sequence accuracy: 81.93020052043471


{'train_acc': [100.0,
  99.99710463836932,
  99.99710463836932,
  100.0,
  100.0,
  100.0],
 'test_acc': [96.96157967243226,
  91.51997550895454,
  89.759681616409,
  95.83652227154447,
  99.44129802540947,
  100.0],
 'val_acc': [98.65076148010886,
  96.69928774103886,
  95.89437720771325,
  97.56789623023916,
  99.35722971799179,
  100.0],
 'train_seq_acc': 99.99420927673867,
 'test_seq_acc': 81.93020052043471,
 'val_seq_acc': 92.33887312525334,
 'train_score': [0.0012191442427505605,
  9.542184659218849e-05,
  0.00047577119569368494,
  0.0003629728998552712,
  0.00022080434270599865,
  6.387599376335974e-05,
  2.979689108205491e-07,
  1.0,
  0.9999710463836933,
  0.9999710463836933,
  1.0,
  1.0,
  1.0],
 'test_score': [1.9249597305217272,
  0.22608713149070242,
  0.6083131103722972,
  0.7564571753749809,
  0.2903091489847405,
  0.04378623184903659,
  6.951734232663811e-06,
  0.9696157967243226,
  0.9151997550712981,
  0.89759681616409,
  0.9583652226971974,
  0.9944129802540946,
  1

#  detection